In [1]:
# import rasterio
# from rasterio import plot
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import asf_search as asf
from datetime import datetime, date, timedelta
from typing import List
from pystac_client import Client, ItemSearch
import geopandas as gpd
from rasterio.crs import CRS
import contextily as cx

In [2]:
# for formatting the datetime object to asfsearch syntax
def datetime2asfsearch(entered_date: datetime) -> str:
    return datetime.strftime(entered_date,'%Y') + '-' + datetime.strftime(entered_date,'%m') + '-' + datetime.strftime(entered_date,'%d') + 'T' + datetime.strftime(entered_date,'%H') + ':' + datetime.strftime(entered_date,'%M') + ':' + datetime.strftime(entered_date,'%S') + 'Z'

In [3]:
def asfsearch2datetime(entered_date: str) -> datetime:
    return datetime.strptime(entered_date, '%Y-%m-%dT%H:%M:%S.%f')

In [4]:
# for searching for sentinel2 and landsat8 data
def hls_search(sensor: str, aoi: Point, date: List[datetime] = None):
    STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'
    api = Client.open(f'{STAC_URL}/LPCLOUD/')
    
    if 'sentinel2' in sensor.lower():
        hls_collections = ['HLSS30.v2.0']
    elif 'landsat8' in sensor.lower():
        hls_collections = ['HLSL30.v2.0']
    
    if date == None:
        search_datetime = [datetime.combine(date.today(), datetime.min.time()), datetime.now()]
    else:
        search_datetime = date
    
    search_params = {
        "collections": hls_collections,
        "bbox": [aoi.x - 1e-5,aoi.y - 1e-5,aoi.x + 1e-5,aoi.y + 1e-5], # list of xmin, ymin, xmax, ymax
        "datetime": search_datetime,
#         "max_items": 1000
    }
    search_hls = api.search(**search_params)
    hls_collection = search_hls.get_all_items()
#     hls_collection = search_hls.get_item_collections()
    d = list(hls_collection)
    
    return d

In [33]:
# for searching sentinel1 data
def asf_search(aoi: Point, date: datetime = None):
    if date == None:
        today = date.today()
        start = str(today) + 'T00:00:00Z'
        end = str(today) + 'T23:59:59Z'
    else:
        start = datetime2asfsearch(date[0])
        end = datetime2asfsearch(date[1])

    opts = {
        'platform': asf.PLATFORM.SENTINEL1,
#         'maxResults': 1000,
        'processingLevel': 'RAW',
        'start': start,
        'end': end
    }
    results = asf.search(**opts)
    
    return results

In [6]:
# find next acquisition date
def acq_search(sensor: str, aoi: Point, date):
    rep = 0
    acq = False
    
    # put arbitrary hard stop at 3 searches (300 days)
    while acq == False and rep < 3:

        if 'landsat8' in sensor.lower():
            results = hls_search('landsat8', aoi, [date + timedelta(days = 100 * rep), date + timedelta(days = 100 * (rep + 1))])
        elif 'sentinel1' in sensor.lower():
            results = asf_search(aoi, [date + timedelta(days = 100 * rep), date + timedelta(days = 100 * (rep + 1))])
        elif 'sentinel2' in sensor.lower():
            results = hls_search('sentinel2', aoi, [date + timedelta(days = 100 * rep), date + timedelta(days = 100 * (rep + 1))])
        
        coords = []
        for i in range(len(results)):
            coords.append(results[i].geometry['coordinates'][0])

        for i in range(len(results)):
            poly = Polygon(coords[i])
            if aoi.within(poly):
                acq = True
                break
            
        if acq:
            if 'landsat8' or 'sentinel2' in sensor.lower():
                next_acq = results[i].properties['start_datetime']
            elif 'sentinel1' in sensor.lower():
                next_acq = results[i].properties['startTime']
            break
        
        else:
            next_acq = 'Search yielded no results'
        rep += 1
    return next_acq

In [22]:
def get_coverage(sensor: List[str], aoi: Point, date: List[datetime] = None) -> List[dict]:
    """
    Sensor: choose sentinel1 and/or landsat8
    AOI: enter coordinates as Point object
    date: leave as none if searching today, else enter time range as datetime tuple: datetime(YYYY,MM,DD)
    """
    freq = {}
    next_acq = {}
    area = {}
    
    for sensor_name in sensor:
        freq[sensor_name] = ''
        next_acq[sensor_name] = ''
        area[sensor_name] = ''
        
        if 'landsat8' in sensor_name.lower():
            results = hls_search('landsat8', aoi, date)
        elif 'sentinel1' in sensor_name.lower():
            results = asf_search(aoi, date)
        elif 'sentinel2' in sensor_name.lower():
            results = hls_search('sentinel2', aoi, date)
        
        coords = []
        for i in range(len(results)):
            coords.append(results[i].geometry['coordinates'][0])

        # find all intersecting polygons and save as list
        tracker = []
        frame = []
        polygon_list = []
        for i in range(len(results)):
            poly = Polygon(coords[i])
            if aoi.within(poly) and i not in frame:
                frame.append(results[i].properties['frameNumber'])
                tracker.append(i)
                polygon_list.append(Polygon(coords[i]))
        
        # calculate frequency
        if not tracker:
            freq[sensor_name] = 'There is no coverage during this time'
            area[sensor_name] = 0
            
        else:
            if len(tracker) == 1:
                try:
                    freq[sensor_name] = 'Only one acquisition on ' + results[i].properties['startTime']
                except:
                    freq[sensor_name] = 'Only one acquisition on ' + results[i].properties['start_datetime']
                    
            else:
                cadence = []
                for i in range(len(tracker) - 1):
                    try:
                        cadence.append(str(asfsearch2datetime(results[tracker[i]].properties['startTime']) - asfsearch2datetime(results[tracker[i + 1]].properties['startTime'])))
                    except:
                        continue
#                         cadence.append(str(asfsearch2datetime(results[tracker[i]].properties['start_datetime']) - asfsearch2datetime(results[tracker[i + 1]].properties['start_datetime'])))
                
                freq[sensor_name] = cadence
        
        # find next acquisition time, if search time is today then returns 'N/A'
        if date == None:
            next_acq[sensor_name] = 'N/A'
            
        else:
            next_acq[sensor_name] = acq_search(sensor_name.lower(), aoi, date[1])
#             if 'sentinel1' in sensor_name.lower():
#                 next_acq[sensor_name] = acq_search('sentinel1', aoi, date[1])
                
#             elif 'sentinel2' in sensor_name.lower():
#                 next_acq[sensor_name] = acq_search('sentinel2', aoi, date[1])
                    
#             elif 'landsat8' in sensor_name.lower():
#                 next_acq[sensor_name] = acq_search('landsat8', aoi, date[1])
                
        if tracker:
            area[sensor_name] = polygon_list[0]
            
            if len(tracker) > 1:
                
                for i in range(len(polygon_list) - 1):
                    area[sensor_name] = area[sensor_name].intersection(polygon_list[i + 1])
         
    return freq, next_acq, area

Ridgecrest coordinates: 35.6225N, 117.6709W (-117.6709, 35.6225)

Wax lake delta: 

Laurentides forest in Canada: 

In [ ]:
freq, next_acq, area = get_coverage(['sentinel1'],Point(-117.6709, 35.6225),[datetime(2022,1,1), datetime(2022,1,14)])

In [ ]:
# print(freq['landsat8'])
# print(next_acq['landsat8'])
print(freq['sentinel1'])
print(next_acq['sentinel1'])

In [ ]:
print(area['landsat8'])
print(area['sentinel1'])

In [28]:
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'maxResults': 10,
    'processingLevel': 'RAW',
    'start': '2022-01-01T00:00:00Z',
    'end': '2022-01-14T23:59:59Z'
}
results = asf.search(**opts)
print(len(results))

10


In [29]:
i = 0
tracker = [0,7]
str(asfsearch2datetime(results[tracker[i]].properties['startTime']) - asfsearch2datetime(results[tracker[i + 1]].properties['startTime']))

'0:02:55'

In [32]:
for i in range(10):
    print(results[i].properties['frameNumber'])

135
130
125
120
115
110
105
100
95
90


In [27]:
results[3].properties

{'beamModeType': 'IW',
 'browse': [],
 'bytes': '1402120249',
 'centerLat': '42.4879',
 'centerLon': '-90.7243',
 'faradayRotation': None,
 'fileID': 'S1A_IW_RAW__0SDV_20220114T235734_20220114T235804_041464_04EE2A_73A5-RAW',
 'fileName': 'S1A_IW_RAW__0SDV_20220114T235734_20220114T235804_041464_04EE2A_73A5.zip',
 'flightDirection': 'ASCENDING',
 'frameNumber': '135',
 'granuleType': 'SENTINEL_1A_FRAME',
 'groupID': 'S1A_IWDV_0135_0141_041464_092',
 'insarStackId': None,
 'md5sum': '66c471fa02359c6c7cc12b929148a9de',
 'offNadirAngle': None,
 'orbit': '41464',
 'pathNumber': '92',
 'perpendicularBaseline': None,
 'platform': 'Sentinel-1A',
 'pointingAngle': None,
 'polarization': 'VV+VH',
 'processingDate': '2022-01-14T23:57:34.000000',
 'processingLevel': 'RAW',
 'sceneName': 'S1A_IW_RAW__0SDV_20220114T235734_20220114T235804_041464_04EE2A_73A5',
 'sensor': 'C-SAR',
 'startTime': '2022-01-14T23:57:34.000000',
 'stopTime': '2022-01-14T23:58:04.000000',
 'temporalBaseline': None,
 'url': 'h

In [ ]:
for i in range(10):
    if results[i].properties['frameNumber'] != '537':
        print(i)
        if results[i].properties['frameNumber'] != '536':
            print(i)
            break

In [ ]:
d[0].geometry

In [ ]:
from shapely.geometry import shape
shape(d[0].geometry)

# Visualization

In [ ]:
area = Polygon(results[2].geometry['coordinates'][0])
test = gpd.GeoDataFrame(geometry = [area], 
                     crs = CRS.from_epsg(4326))
test.boundary.plot()

In [ ]:
df_l8 = gpd.GeoDataFrame(geometry = [area['landsat8']], 
                     crs = CRS.from_epsg(4326))
df_wm = df_l8.to_crs(epsg=3857)

In [ ]:
ax = df_wm.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
cx.add_basemap(ax, zoom = 10)

In [ ]:
df_l8.boundary.plot()

In [ ]:
df_s1 = gpd.GeoDataFrame(geometry = [area['sentinel1']], 
                     crs = CRS.from_epsg(4326))

In [ ]:
df_s1.plot()

In [ ]:
fig, ax = plt.subplots()

df_l8.plot(ax =ax, alpha=.5)
df_s1.plot(ax = ax, alpha=.5)

In [ ]:
results[0].properties

In [ ]:
Point(-121.5, 34.95)
STAC_URL = 'https://earth-search.aws.element84.com/v0'
api = Client.open(STAC_URL)
url_collections = ['sentinel-s2-l2a-cogs']
search_params = {"collections": url_collections,
#                  "bbox": [-121.5, 34.95, -120.2, 36.25], # list of xmin, ymin, xmax, ymax
                 "datetime": [datetime(2016,1,1), datetime(2016,2,1)],
                 "max_items": 500}
search_hls = api.search(**search_params)
hls_collection = search_hls.get_all_items()
d = list(hls_collection)